## 05. Analyze 

## Setting

In [2]:
import pandas as pd 
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from review_analyzer import *

# rc('font', family='AppleGothic') # Mac 
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False

In [3]:
data_path = "data/naver_review_preprocessing.csv"
data = pd.read_csv(data_path, parse_dates=["date"], encoding="CP949")
data.head(5)

,reviewer,review,date,weekday
0,shooooooo,종류도 알차게 많고 다 맛있어요,2024-06-07,금요일
1,쥴리08,커피랑 브런치크림파스타 먹었어요 오늘따라 파스타가 불어서나왔더라구요 역시 신라 커피...,2024-05-23,목요일
2,illiiilillil,직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금...,2024-05-19,일요일
3,DEAN KIL,비싸지만 좋은곳,2024-05-04,토요일
4,이진욱쨩,항상 친절하시고 음식도맛있네요,2024-04-21,일요일


## 1. 프롬프트

In [ ]:
# 내가 원하는 출력 결과
{
    "맛": {"긍정": [], "부정": []},
    "서비스": {"긍정": [], "부정": []},
    "환경": {"긍정": [], "부정": []},
    "기타": {"긍정": [], "부정": []}
}

In [11]:
from pydantic import BaseModel, Field
from typing import Dict, List, Optional 
from langchain_core.output_parsers import JsonOutputParser

class ActionModel(BaseModel):
    positive: List[str] = Field(description="주제에 대한 간결한 설명")
    negative: List[str] = Field(description="해시태그 형식의 키워드(2개 이상)")

parser = JsonOutputParser(pydantic_object=ActionModel)
parser

JsonOutputParser(pydantic_object=<class '__main__.Topic'>)

In [15]:
from pydantic import BaseModel, Field
from typing import List

class ActionModel(BaseModel):
    positive: List[str] = Field(description="주제에 대한 간결한 설명")
    negative: List[str] = Field(description="해시태그 형식의 키워드(2개 이상)")

mydict = {
    "positive": ["A", "B", "C"],
    "negative": ["D", "E", "F"]
}
ActionModel(**mydict)

ValidationError: 1 validation error for ActionModel
negative
  Input should be a valid list [type=list_type, input_value='hello', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/list_type

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)
prompt.partial()

PromptTemplate(input_variables=['sentence'], template='# INSTRUCTION\n- 당신은 긍/부정 분류기입니다.\n- 각 대상 \'만족도\', \'맛\', \'서비스\', \'가격\'에 대한 평가가 긍정적인지 부정적인지를 분류하세요.\n- 대상에 대한 평가가 없는 경우 \'-\'을 표시하세요.\n- 예시를 보고 결과를 다음과 같은 딕셔너리 형식으로 출력하세요:\n\n    "만족도": "긍정/부정/-",\n    "맛": "긍정/부정/-",\n    "서비스": "긍정/부정/-",\n    "가격": "긍정/부정/-"\n\n# SENTENCE: {sentence}\n')

In [4]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
parser.get_format_instructions()

'Return a JSON object.'

In [5]:
template = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 각 대상 '만족도', '맛', '서비스', '가격'에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 대상에 대한 평가가 없는 경우 '-'을 표시하세요.
- 예시를 보고 결과를 다음과 같은 딕셔너리 형식으로 출력하세요:

    "만족도": "긍정/부정/-",
    "맛": "긍정/부정/-",
    "서비스": "긍정/부정/-",
    "가격": "긍정/부정/-"

# SENTENCE: {sentence}
"""

In [90]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 {sentence}에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- JSON 형식으로 출력하세요.
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(category):
    return {"sentence": RunnablePassthrough() , "category": RunnablePassthrough(lambda _: category)}

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke(review)


{'taste': {'feedback': '긍정'},
 'service': {'feedback': '긍정'},
 'price': {'feedback': '긍정'}}

In [109]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 SENTENCE에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- 다음 KEYS를 참고하여 JSON 형식으로 출력하세요.

# KEYS:
- category <{category}반응과 관련된 대상>
- action <반응>

# SENTENCE: {sentence}
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(text):
    return RunnablePassthrough.assign(category = lambda x: text)

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke({"sentence": review})


{'taste': {'category': '맛', 'action': '긍정'},
 'service': {'category': '직원 및 음식', 'action': '긍정'},
 'price': {'category': '가격', 'action': '-'}}

In [94]:
cat = "맛"
add_runnable = RunnablePassthrough.assign(taste = lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐', 'taste': '안뇽'}

In [101]:
cat = "맛"
add_runnable = RunnablePassthrough(lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐'}

In [ ]:
next